In [1]:
# !powershell -command "& { Invoke-WebRequest -Uri 'https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt' -OutFile 'edgar_allan_poe.txt' }"
# !powershell -command "& { Invoke-WebRequest -Uri 'https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt' -OutFile 'robert_frost.txt' }"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [3]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [4]:
# !head edgar_allan_poe.txt
with open('edgar_allan_poe.txt', 'r') as file:
    for i in range(10):  # Adjust the number of lines as needed
        line = file.readline()
        print(line.strip())

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
â€‰
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [5]:
with open('robert_frost.txt', 'r') as file:
    for i in range(10):  # Adjust the number of lines as needed
        line = file.readline()
        print(line.strip())

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth;

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [6]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)
        
# input_texts
# labels

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [7]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)
len(Ytrain), len(Ytest)

(1618, 540)

In [8]:
train_text[:5]

['he never said much after we were married',
 'hes been in a glass case as you may say',
 'like no she aint come back from kiting yet',
 'you couldnt call it living for it aint',
 'plumes till they trailed in the dust ']

In [9]:
Ytrain[:5]

[1, 1, 1, 1, 0]

In [10]:
idx = 1
word2idx = {'<unk>': 0}
word2idx

{'<unk>': 0}

In [11]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1
# word2idx
# len(word2idx)

In [12]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [13]:
train_text_int[100:105]

[[224, 274, 376, 151, 322, 377],
 [298, 1, 90, 235, 37, 378, 379],
 [23, 73, 332, 135, 16, 93, 68, 380, 161, 381],
 [54, 175, 382, 383, 32, 109, 384, 19, 68, 380],
 [48, 11, 12, 215, 77, 64, 20, 146, 12, 215]]

In [14]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [53]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1

      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1
      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [70]:
# pi0
# A0
# Ytrain

In [55]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [56]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [58]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.3331273176761434, 0.6668726823238567)

In [59]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]
      
      # update last_idx
      last_idx = idx
    
    return logprob
  
  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [60]:
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [61]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9993819530284301


In [62]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8185185185185185


In [63]:
from sklearn.metrics import confusion_matrix, f1_score


In [64]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 538,    1],
       [   0, 1079]], dtype=int64)

In [65]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[129,  54],
       [ 44, 313]], dtype=int64)

In [66]:
f1_score(Ytrain, Ptrain)

0.999536822603057

In [67]:
f1_score(Ytest, Ptest)

0.8646408839779005